In [4]:
import pandas as pd
import json
import math as m
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import model_selection
import collections
import re
import networkx as nx
import nltk
from nltk.util import ngrams
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import TruncatedSVD
# from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
# from sklearn.base import BaseEstimator, TransformerMixin

In [70]:
data_json = []
with open('data/dblp-ref-0.json') as f:
    for line in f:
        data_json.append(json.loads(line))
with open('data/dblp-ref-1.json') as f:
    for line in f:
        data_json.append(json.loads(line))
with open('data/dblp-ref-2.json') as f:
    for line in f:
        data_json.append(json.loads(line))
with open('data/dblp-ref-3.json') as f:
    for line in f:
        data_json.append(json.loads(line))

In [71]:
df = pd.DataFrame(data_json)

In [73]:
df = df[df['abstract'].notnull()]

In [8]:
parentdomain = {
    "coding theory" : "mathematical foundations",
    "game theory" : "mathematical foundations",
    "graph theory" : "mathematical foundations",
    "mathematical logic" : "mathematical foundations",
    "boolean logic" : "mathematical foundations",
    "number theory" : "mathematical foundations",
    "mathematical foundations" : "mathematical foundations",
    "algorithms" : "algorithms and data structures",
    "data structures" : "algorithms and data structures",
    "algorithms and data structures" : "algorithms and data structures",
    "automated reasoning" : "artificial intelligence",
    "computer vision" : "artificial intelligence",
    "machine learning" : "artificial intelligence",
    "evolutionary computing" : "artificial intelligence",
    "natural language processing" : "artificial intelligence",    
    "artificial intelligence" : "artificial intelligence",
    "neural network" : "artificial intelligence",
    "speech processing" : "artificial intelligence",
    "information retrieval" : "artificial intelligence",
    "data mining" : "artificial intelligence",
    "robotics" : "artificial intelligence",
    "networking" : "communication and security",
    "computer networks" : "communication and security",
    "communication networks" : "communication and security",
    "cryptography" : "communication and security",
    "communication and security" : "communication and security",
    "network protocols" : "communication and security",
    "network attacks" : "communication and security",
    "operating systems" : "computer architecture",
    "computer architecture" : "computer architecture",
    "image processing" : "computer graphics",
    "information visualization" : "computer graphics",
    "visualization" : "computer graphics",
    "visualisation" : "computer graphics",
    "computer graphics" : "computer graphics",
    "parallel computing" : "concurrent, parallel, and distributed systems",
    "concurrency" : "concurrent, parallel, and distributed systems",
    "distributed computing" : "concurrent, parallel, and distributed systems",
    "concurrent, parallel, and distributed systems" : "concurrent, parallel, and distributed systems",
    "relational databases" : "databases",
    "structured storage" : "databases",
    "databases" : "databases",
    "compiler theory" : "programming languages and compilers",
    "programming language" : "programming languages and compilers",
    "programming language pragmatics" : "programming languages and compilers",
    "programming language theory" : "programming languages and compilers",
    "formal semantics" : "programming languages and compilers",
    "type theory" : "programming languages and compilers",
    "programming languages and compilers" : "programming languages and compilers",
    "computational science" : "scientific computing",
    "numerical analysis" : "scientific computing",
    "symbolic computation " : "scientific computing",
    "computational physics" : "scientific computing",
    "computational chemistry" : "scientific computing",
    "bioinformatics" : "scientific computing",
    "computational biology" : "scientific computing",
    "computational neuroscience" : "scientific computing",
    "scientific computing" : "scientific computing",
    "formal methods" : "software engineering",
    "algorithm design" : "software engineering",
    "computer programming" : "software engineering",
    "human–computer interaction" : "software engineering",
    "reverse engineering" : "software engineering",
    "software engineering" : "software engineering",
    "automata theory " : "theory of computation",
    "computability theory" : "theory of computation",
    "computational complexity theory" : "theory of computation",
    "quantum computing" : "theory of computation",
    "theory of computation" : "theory of computation",   
}

In [9]:
df_copy = df.copy()
def getdomainlist(row):
    abst = row['abstract'] if type(row['abstract']) != float else  ""
    venue = row['venue'] if type(row['venue']) != float else  ""
    
    text = abst + " " + row['title'] + " " + venue
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    tokens = [token for token in text.split(" ") if token != ""]
    output1 = list(ngrams(tokens, 1))
    output2 = list(ngrams(tokens, 2))
    output3 = list(ngrams(tokens, 3))
    output = []
    for words in output1:
        output.append(' '.join(words))
    for words in output2:
        output.append(' '.join(words))
    for words in output3:
        output.append(' '.join(words))
    domainlist = set()
    for word in output:
        if word in parentdomain:
            domainlist.add(parentdomain[word])
    if len(domainlist) > 0:
        return list(domainlist)[0]
    else:
        return ""

df_copy['domain'] = df_copy.apply(getdomainlist, axis = 1)

In [10]:
dffinal = df_copy[df_copy['domain'].map(lambda x: len(x) >0)]

In [11]:
dffinal['text'] = dffinal[['abstract', 'title', 'venue']].apply(lambda x: ' '.join(x), axis=1)

/jet/var/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
X_train, X_test, y_train, y_test = train_test_split(dffinal['text'].head(40000), dffinal['domain'].head(40000), random_state = 0)

pipeline = Pipeline([('vect', CountVectorizer(stop_words='english')),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge'))
                     ])

parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    # 'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    # 'tfidf__use_idf': (True, False),
    # 'tfidf__norm': ('l1', 'l2'),
    'clf__max_iter': (5,),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    # 'clf__max_iter': (10, 50, 80),
}

grid_search = GridSearchCV(pipeline, parameters, cv=5,n_jobs=10,
                               verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:  1.9min
[Parallel(n_jobs=10)]: Done 120 out of 120 | elapsed:  8.3min finished
/jet/var/python/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        ...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=10,
       param_grid={'vect__max_df': (0.5, 0.75, 1.0), 'vect__ngram_range': ((1, 1), (1, 2)), 'clf__max_iter': (5,), 'clf__alpha': (1e-05, 1e-06), 'clf__penalty': ('l2', 'elasticnet')},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(dffinal['text'].head(50000), dffinal['domain'].head(50000), random_state = 0)
# class TextSelector(BaseEstimator, TransformerMixin):
#     def __init__(self, field):
#         self.field = field
#     def fit(self, X, y=None):
#         return self
#     def transform(self, X):
#         return X[self.field]

# def Tokenizer(str_input):
#     words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
#     porter_stemmer=nltk.PorterStemmer()
#     words = [porter_stemmer.stem(word) for word in words]
#     return words

# classifier = Pipeline([
#             ('colext', TextSelector('text')),
#             ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words="english",
#                      min_df=.0025, max_df=0.25, ngram_range=(1,3))),
#             ('svd', TruncatedSVD(algorithm='randomized', n_components=300)), #for XGB
#     ('clf', XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1)),
# #    ('clf', RandomForestClassifier()),
#     ])

# parameters = {
# #     'vect__max_df': (0.5, 0.75, 1.0),
# #     # 'vect__max_features': (None, 5000, 10000, 50000),
# #     'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
# #     # 'tfidf__use_idf': (True, False),
# #     # 'tfidf__norm': ('l1', 'l2'),
# #     'clf__max_iter': (5,),
# #     'clf__alpha': (0.00001, 0.000001),
# #     'clf__penalty': ('l2', 'elasticnet'),
#     # 'clf__max_iter': (10, 50, 80),
# }

# grid_search = GridSearchCV(clas, parameters, cv=5,n_jobs=10,
#                                verbose=1)
# grid_search.fit(X_train, y_train)

In [13]:
df['text'] = df[['abstract', 'title', 'venue']].apply(lambda x: ' '.join(x), axis=1)

In [14]:
len(df)

753646

In [15]:
answer = grid_search.predict(df['text'].values.tolist())

In [16]:
df['domain'] = answer.tolist()

In [75]:
pidtod = {}
def domains(row):
    pidtod[row['id']] = row['domain']
        
x = df.apply(domains, axis = 1)

KeyError: ('domain', 'occurred at index 0')

In [76]:
pidtoyear = {}
def getyear(row):
    pidtoyear[row['id']] = row['year']
        
x = df.apply(getyear, axis = 1)

In [77]:
paperid = df['id'].values.tolist()
papernames = df['title'].values.tolist()
paperref = df['references'].values.tolist()
paperidmap = {}
paperidtoname = {}
paperindextoid = {}
g=nx.DiGraph()
for index, i in enumerate(paperid):
    paperidmap[i] = index
    paperindextoid[index] = i
    paperidtoname[i] = papernames[index]
    
ptop = {}
for i in range(len(paperid)):
    ptop[paperid[i]] = paperref[i]


# for key in ptop.keys():
#     t = set()
#     if type(ptop[key]) == list:
#         for i in ptop[key]:
#             if i in paperidmap and pidtod[i] == pidtod[key]:
#                 g.add_edge(paperidmap[key],paperidmap[i])

In [19]:
pr = nx.pagerank(g, alpha=0.85)

In [20]:
answer = sorted(pr.items(), key=lambda kv: kv[1], reverse = True)

In [21]:
for elem in answer[:100]:
    print(paperidtoname[paperindextoid[elem[0]]])

Genetic Algorithms in Search, Optimization and Machine Learning
The Design and Analysis of Computer Algorithms
A method for obtaining digital signatures and public-key cryptosystems
Genetic programming: on the programming of computers by means of natural selection
Christopher D. Manning and Hinrich Schutze. Foundations of Statistical Natural Language Processing . MIT Press, 2000. ISBN 0-262-13360-1. 620 pp. $64.95/£44.95 (cloth).
Handbook of Applied Cryptography
Minimizing the real functions of the ICEC'96 contest by differential evolution
Term-weighting approaches in automatic text retrieval
Differential Evolution – A Simple and Efficient Heuristic for Global Optimization over Continuous Spaces
Perceived usefulness, perceived ease of use, and user acceptance of information technology
Integer and combinatorial optimization
Solving multiclass learning problems via error-correcting output codes
Network flows: theory, algorithms, and applications
Boosting the margin: a new explanation for

In [23]:
top100papers = answer[:100]


In [78]:
pidtocitationyears = {}
for p in paperid:
    year = pidtoyear[p]
    if type(ptop[p]) == list: 
        for rp in ptop[p]:
            if rp in paperidmap:
                rid = paperidmap[rp]
                if rid not in pidtocitationyears:
                    pidtocitationyears[rid] = {}

                if year not in pidtocitationyears[rid]:
                    pidtocitationyears[rid][year] = 1
                else:
                    pidtocitationyears[rid][year] += 1


In [80]:
for p in top100papers:
    print(p[0], pidtocitationyears[p[0]])

349209 {2017: 116, 2013: 621, 2012: 649, 2011: 804, 2008: 834, 2001: 409, 2004: 581, 2005: 711, 2000: 358, 2010: 825, 2006: 745, 1998: 344, 1999: 296, 2002: 441, 1991: 22, 1992: 33, 2014: 533, 2009: 898, 1990: 18, 1996: 154, 2003: 517, 1997: 212, 2007: 873, 1994: 107, 2015: 428, 1993: 54, 2016: 326, 1995: 121, 1989: 5, 1988: 2}
354312 {1987: 99, 1985: 76, 1993: 109, 2000: 91, 2002: 80, 2007: 69, 2012: 53, 1977: 34, 2014: 47, 1986: 71, 1983: 73, 2015: 40, 1998: 97, 2011: 60, 1996: 86, 1979: 60, 1988: 93, 1984: 86, 2013: 44, 1997: 97, 1994: 105, 1976: 37, 1990: 102, 2010: 82, 1980: 63, 2006: 73, 1992: 108, 1981: 70, 2003: 83, 2008: 75, 1995: 77, 2017: 11, 1989: 100, 2001: 86, 1978: 48, 1982: 45, 2009: 81, 2005: 98, 1999: 97, 2004: 87, 2016: 37, 1991: 106, 1975: 10, 1973: 1, 1974: 2}
711061 {2003: 145, 2012: 194, 2005: 246, 2011: 222, 2013: 189, 2008: 229, 2009: 244, 2007: 231, 1992: 35, 1996: 52, 1990: 35, 2002: 119, 1998: 92, 2004: 207, 1985: 29, 2006: 229, 2014: 166, 1995: 50, 1999: 12